In [1]:
import os

import scutils_cnb_clean
from scutils_cnb_clean import *
np.random.seed(1342)

from mpl_toolkits.mplot3d.art3d import LineCollection
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors

import rpy2
import anndata2ri

import scanpy
import anndata

import gseapy

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


In [2]:
%matplotlib inline

In [3]:
working_dir = '/Users/burdziac/Documents/PDAC/notebooks_final_June2021/'

# Functions

In [4]:
def plot_G(tsne, G,ax,
           alpha_G=0.7, color_G='r',s=10,t='k',cmap='viridis',colorbar=False):
    
    # generate collection of all edges (assumes symmetric)
    segments = [tsne[[i, j],:] for (i, j) in G.todok().keys() if i < j]
    ax.add_collection(LineCollection(segments, colors=color_G, alpha=alpha_G))

    if colorbar:
        plt.colorbar()
    plt.axis("off")
    ax.scatter(tsne[:,0],tsne[:,1],s=s,c=t,cmap=cmap,edgecolor='k',zorder=10,vmin=-5,vmax=5)
    return ax

#using script from Joe Chan
def convert_genes(genes_to_map,direction='h2m'):
    pd.DataFrame(genes_to_map,columns=['Genes']).to_csv("scripts/script_intermediates/genes_tmp.csv")
    command = "Rscript scripts/ConvertHuman2MouseGenes_Joe_CNBEdit.R {}".format(direction)
    os.system(command)
    output = pd.read_csv("scripts/script_intermediates/converted_tmp.csv",index_col=0)
    os.system("rm scripts/script_intermediates/converted_tmp.csv")
    return output['x'].values

def plot_G2(tsne, G,ax,
           alpha_G=0.7, color_G='r',s=10,t='k',cmap='viridis',colorbar=False):
    
    # generate collection of all edges (assumes symmetric)
    segments = [tsne[[i, j],:] for (i, j) in G.todok().keys() if i < j]
    ax.add_collection(LineCollection(segments, colors=color_G, alpha=alpha_G))

    if colorbar:
        plt.colorbar()
    plt.axis("off")
    ax.scatter(tsne[:,0],tsne[:,1],s=s,c=t,cmap=cmap,edgecolor='k',zorder=10)
    return ax

# Hard-coded Metadata

In [5]:
stage_dict = {'DACD345_CD45_PLUS':'T5/IL33KD',
    'DACD346-CD45+':'T4/IL33KD',
 'DACD347_CD45_PLUS':'T5/IL33KD',
 'DACD349_CD45_PLUS':"T5",
 'DACD350-CD45+':'T4',
 'DACD351-CD45+':'T3',
 'DACE604-CD45_pos':"T4/IL33KD",
 'DACE605-CD45_pos':'T4/IL33KD',
 'DACE621-CD45_pos':"T4",
 'E607-CD45':'T5/IL33KD',
 'E610-CD45':"T5",
 'E614-CD45':"T5/IL33KD"}

In [6]:
cohort_dict = {'DACD345_CD45_PLUS':'C1',
    'DACD346-CD45+':'C1',
 'DACD347_CD45_PLUS':'C1',
 'DACD349_CD45_PLUS':'C1',
 'DACD350-CD45+':'C1',
 'DACD351-CD45+':'C1',
 'DACE604-CD45_pos':"C2",
 'DACE605-CD45_pos':'C2',
 'DACE621-CD45_pos':"C2",
 'E607-CD45':'C2',
 'E610-CD45':"C2",
 'E614-CD45':"C2"}

In [7]:
color_dict= {
"T3":"#EED04D",
"T4":"#1D89E3",
"T5":"#F8BADE",
"T4/IL33KD":"#004b8a",
"T5/IL33KD":"#804e6b"}

# Set up R Environment

In [8]:
anndata2ri.activate()

In [9]:
%load_ext rpy2.ipython

In [10]:
%%R
library(miloR)
library(igraph)
library(ggplot2)

R[write to console]: Loading required package: edgeR

R[write to console]: Loading required package: limma

R[write to console]: 
Attaching package: ‘igraph’


R[write to console]: The following object is masked from ‘package:miloR’:

    graph


R[write to console]: The following objects are masked from ‘package:stats’:

    decompose, spectrum


R[write to console]: The following object is masked from ‘package:base’:

    union


R[write to console]: Need help? Try Stackoverflow: https://stackoverflow.com/tags/ggplot2



# Preparation to Run Milo

## Load Data in Python 

In [11]:
sc = load_sc(working_dir+"saved_analyses/CD45Plus_PerturbationSubset_T5.pickle")

## Set Up adata Object & Pass to R

In [12]:
cell_barcodes = ["{}_{}".format(sc.cell_ids[i],sc.sample_ids[i]) for i in range(sc.data.shape[0])]
condition = [stage_dict[i] for i in sc.sample_ids]
cohort = [cohort_dict[i] for i in sc.sample_ids]
obs = pd.DataFrame(dict(batch=sc.sample_ids,total_molecules=sc.lib_size.astype(np.int32),
                       condition=condition,cohort=cohort,celltype=sc.communities_refined,
                  cluster = sc.communities_coarse),index=cell_barcodes,)
var = pd.DataFrame(index=list(sc.data_normalized_libsize))

#obsm = {"X_pca":sc.pcs_corrected,
#       "X_tsne":sc.tsne_corrected,"X_umap":sc.tsne_corrected}

#not using corrected PCs
obsm = {"X_pca":sc.pc_log,
       "X_tsne":sc.tsne_corrected,"X_umap":sc.tsne_corrected}

num_PCs = sc.npca_log

In [13]:
adata = anndata.AnnData(X=np.log(sc.data_normalized_libsize.values+.1),#csr_matrix(sc.data_normalized.values.astype(np.float32)), 
                        obs=obs, 
                        var=var, 
                        uns=None, 
                        obsm=obsm, 
                        varm=None, layers=None, raw=None, dtype='float32', 
                        shape=None, filename=None, filemode=None, asview=False, 
                        obsp=None, varp=None, oidx=None, vidx=None)

In [14]:
%%R -i adata
adata

class: SingleCellExperiment 
dim: 13408 6774 
metadata(0):
assays(1): X
rownames(13408): WTIP HIST1H2AN ... GBP7 MREG
rowData names(0):
colnames(6774): 120703423794532_DACD349_CD45_PLUS
  120703424093427_DACD349_CD45_PLUS ... 241114607970611_E614-CD45
  241114608004012_E614-CD45
colData names(6): batch total_molecules ... celltype cluster
reducedDimNames(3): PCA TSNE UMAP
mainExpName: NULL
altExpNames(0):


In [15]:
%%R 
milo <- Milo(adata)
milo

class: Milo 
dim: 13408 6774 
metadata(0):
assays(1): X
rownames(13408): WTIP HIST1H2AN ... GBP7 MREG
rowData names(0):
colnames(6774): 120703423794532_DACD349_CD45_PLUS
  120703424093427_DACD349_CD45_PLUS ... 241114607970611_E614-CD45
  241114608004012_E614-CD45
colData names(6): batch total_molecules ... celltype cluster
reducedDimNames(3): PCA TSNE UMAP
mainExpName: NULL
altExpNames(0):
nhoods dimensions(2): 1 1
nhoodCounts dimensions(2): 1 1
nhoodDistances dimension(1): 0
graph names(0):
nhoodIndex names(1): 0
nhoodExpression dimension(2): 1 1
nhoodReducedDim names(0):
nhoodGraph names(0):
nhoodAdjacency dimension(2): 1 1


In [16]:
%%R -i num_PCs
num_PCs

[1] 58


In [17]:
%%R 
milo <- buildGraph(milo,
  transposed = TRUE,
  get.distance = FALSE,
  reduced.dim = "PCA"
)

R[write to console]: Constructing kNN graph with k:10



In [18]:
%%R
milo

class: Milo 
dim: 13408 6774 
metadata(0):
assays(1): X
rownames(13408): WTIP HIST1H2AN ... GBP7 MREG
rowData names(0):
colnames(6774): 120703423794532_DACD349_CD45_PLUS
  120703424093427_DACD349_CD45_PLUS ... 241114607970611_E614-CD45
  241114608004012_E614-CD45
colData names(6): batch total_molecules ... celltype cluster
reducedDimNames(3): PCA TSNE UMAP
mainExpName: NULL
altExpNames(0):
nhoods dimensions(2): 1 1
nhoodCounts dimensions(2): 1 1
nhoodDistances dimension(1): 0
graph names(1): graph
nhoodIndex names(1): 0
nhoodExpression dimension(2): 1 1
nhoodReducedDim names(0):
nhoodGraph names(0):
nhoodAdjacency dimension(2): 1 1


# Run Milo

## Prepare Design Matrix for Milo

In [19]:
#define dataframe to input in milo (at least cells x independent variable)
design_df = obs[["batch", "condition",'cohort']]
design_df.drop_duplicates(inplace=True)
design_df.index = design_df['batch']
design_df

/Users/burdziac/Documents/bin/miniconda2/envs/MyEnv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,batch,condition,cohort
batch,,,
DACD349_CD45_PLUS,DACD349_CD45_PLUS,T5,C1
DACD347_CD45_PLUS,DACD347_CD45_PLUS,T5/IL33KD,C1
E607-CD45,E607-CD45,T5/IL33KD,C2
E610-CD45,E610-CD45,T5,C2
E614-CD45,E614-CD45,T5/IL33KD,C2


## Run Milo in R

In [20]:
%%R -i design_df -o DA_results
## Define neighbourhoods
milo <- makeNhoods(milo, d=num_PCs, refined = TRUE)

## Count cells in neighbourhoods
milo <- countCells(milo, meta.data = data.frame(colData(milo)), sample="batch")

## Calculate distances between cells in neighbourhoods
## for spatial FDR correction
milo <- calcNhoodDistance(milo, d=num_PCs)

## Test for differential abundance set ~ <confounding_variable>  + <independent variable>

#correcting for confounding variable (key of that variable in original adata.obs)
#DA_results <- testNhoods(milo, design = ~ confounding_variable + independent_variable , design.df = design_df)

#without correcting for confounding variable
DA_results <- testNhoods(milo, design = ~ cohort + condition , design.df = design_df)

R[write to console]: Checking valid object

R[write to console]: Checking meta.data validity

R[write to console]: Counting cells in neighbourhoods

R[write to console]: Using TMM normalisation

R[write to console]: Performing spatial FDR correction withk-distance weighting



In [21]:
%%R
milo

class: Milo 
dim: 13408 6774 
metadata(0):
assays(1): X
rownames(13408): WTIP HIST1H2AN ... GBP7 MREG
rowData names(0):
colnames(6774): 120703423794532_DACD349_CD45_PLUS
  120703424093427_DACD349_CD45_PLUS ... 241114607970611_E614-CD45
  241114608004012_E614-CD45
colData names(6): batch total_molecules ... celltype cluster
reducedDimNames(3): PCA TSNE UMAP
mainExpName: NULL
altExpNames(0):
nhoods dimensions(2): 6774 486
nhoodCounts dimensions(2): 486 5
nhoodDistances dimension(1): 486
graph names(1): graph
nhoodIndex names(1): 486
nhoodExpression dimension(2): 1 1
nhoodReducedDim names(0):
nhoodGraph names(0):
nhoodAdjacency dimension(2): 1 1


In [22]:
DA_results.sort_values('FDR')

,logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR
470,-2.697832,12.579316,10.266511,0.001399,0.169490,470.0,0.148498
30,-2.598729,12.749513,10.457588,0.001263,0.169490,30.0,0.148498
43,-4.364352,11.944076,12.025847,0.000548,0.169490,43.0,0.148498
238,-4.912691,11.341737,9.700182,0.001896,0.169490,238.0,0.148498
352,-4.695478,11.247034,9.505751,0.002106,0.169490,352.0,0.148498
...,...,...,...,...,...,...,...
321,0.036303,11.121197,0.000499,0.982179,0.988280,321.0,0.988261
411,-0.028774,11.071016,0.000553,0.981236,0.988280,411.0,0.988261
408,0.014325,10.969503,0.000133,0.990797,0.994891,408.0,0.994835
465,0.007962,11.734249,0.000060,0.993812,0.995861,465.0,0.995606


In [23]:
%%R -o DA_results_annot -o DA_results_annot_coarse
DA_results_annot <- annotateNhoods(milo, DA_results, coldata_col = "celltype")
DA_results_annot_coarse <- annotateNhoods(milo, DA_results, coldata_col = "cluster")

R[write to console]: Converting celltype to factor...

R[write to console]: Converting cluster to factor...



In [24]:
DA_results_annot.to_csv(working_dir+"final_outputs/Figure6/DAResultsAnnot_T5Immune.csv",index=True,header=True)